In [1]:
import torch
import numpy as np
import numpy.random as rd
import sys


class CueAccumulationDataset(torch.utils.data.Dataset):
    """Adapted from the original TensorFlow e-prop implemation from TU Graz, available at https://github.com/IGITUGraz/eligibility_propagation"""

    def __init__(self, args, type):
      
        n_cues     = 7
        f0         = 40
        t_cue      = 100
        t_wait     = 1200
        n_symbols  = 4
        p_group    = 0.3
        
        self.dt         = 1e-3
        self.t_interval = 150
        self.seq_len    = n_cues*self.t_interval + t_wait
        self.n_in       = 40
        self.n_out      = 2    # This is a binary classification task, so using two output units with a softmax activation redundant
        n_channel       = self.n_in // n_symbols
        prob0           = f0 * self.dt
        t_silent        = self.t_interval - t_cue
        
        if (type == 'train'):
            length = args.train_len
        else:
            length = args.test_len
            
    
        # Randomly assign group A and B
        prob_choices = np.array([p_group, 1 - p_group], dtype=np.float32)
        idx = rd.choice([0, 1], length)
        probs = np.zeros((length, 2), dtype=np.float32)
        # Assign input spike probabilities
        probs[:, 0] = prob_choices[idx]
        probs[:, 1] = prob_choices[1 - idx]
    
        cue_assignments = np.zeros((length, n_cues), dtype=np.int)
        # For each example in batch, draw which cues are going to be active (left or right)
        for b in range(length):
            cue_assignments[b, :] = rd.choice([0, 1], n_cues, p=probs[b])
    
        # Generate input spikes
        input_spike_prob = np.zeros((length, self.seq_len, self.n_in))
        t_silent = self.t_interval - t_cue
        for b in range(length):
            for k in range(n_cues):
                # Input channels only fire when they are selected (left or right)
                c = cue_assignments[b, k]
                input_spike_prob[b, t_silent+k*self.t_interval:t_silent+k*self.t_interval+t_cue, c*n_channel:(c+1)*n_channel] = prob0
    
        # Recall cue and background noise
        input_spike_prob[:, -self.t_interval:, 2*n_channel:3*n_channel] = prob0
        input_spike_prob[:, :, 3*n_channel:] = prob0/4.
        input_spikes = generate_poisson_noise_np(input_spike_prob)
        self.x = torch.tensor(input_spikes).float()
    
        # Generate targets
        target_nums = np.zeros((length, self.seq_len), dtype=np.int)
        target_nums[:, :] = np.transpose(np.tile(np.sum(cue_assignments, axis=1) > int(n_cues/2), (self.seq_len, 1)))
        self.y = torch.tensor(target_nums).long()
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.x[index], self.y[index]



def setup(args):
    args.cuda = not args.cpu and torch.cuda.is_available()
    if args.cuda:
        print("=== The available CUDA GPU will be used for computations.")
        device = torch.cuda.current_device()
    else:
        device = torch.device('cpu')
    
    kwargs = {'num_workers': 0, 'pin_memory': True} if args.cuda else {}

    if args.dataset == "cue_accumulation":
        print("=== Loading cue evidence accumulation dataset...")
        (train_loader, traintest_loader, test_loader) = load_dataset_cue_accumulation(args, kwargs)
    else:
        print("=== ERROR - Unsupported dataset ===")
        sys.exit(1)
        
    print("Training set length: "+str(args.full_train_len))
    print("Test set length: "+str(args.full_test_len))
    
    return (device, train_loader, traintest_loader, test_loader)


def load_dataset_cue_accumulation(args, kwargs):

    trainset = CueAccumulationDataset(args,"train")
    testset  = CueAccumulationDataset(args,"test")

    train_loader     = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size,      shuffle=args.shuffle, **kwargs)
    traintest_loader = torch.utils.data.DataLoader(trainset, batch_size=args.test_batch_size, shuffle=False       , **kwargs)
    test_loader      = torch.utils.data.DataLoader(testset , batch_size=args.test_batch_size, shuffle=False       , **kwargs)
    
    args.n_classes      = trainset.n_out
    args.n_steps        = trainset.seq_len
    args.n_inputs       = trainset.n_in
    args.dt             = trainset.dt
    args.classif        = True
    args.full_train_len = len(trainset)
    args.full_test_len  = len(testset)
    args.delay_targets  = trainset.t_interval
    args.skip_test      = False
    
    return (train_loader, traintest_loader, test_loader)


def generate_poisson_noise_np(prob_pattern, freezing_seed=None):
    if isinstance(prob_pattern, list):
        return [generate_poisson_noise_np(pb, freezing_seed=freezing_seed) for pb in prob_pattern]

    shp = prob_pattern.shape

    if not(freezing_seed is None): rng = rd.RandomState(freezing_seed)
    else: rng = rd.RandomState()

    spikes = prob_pattern > rng.rand(prob_pattern.size).reshape(shp)
    return spikes


In [7]:
import argparse


def args():
    parser = argparse.ArgumentParser(
        description="Train a reservoir based SNN on biosignals"
    )

    parser.add_argument('--cpu', action='store_true', default=True, help='Disable CUDA training and run training on CPU')
    parser.add_argument('--dataset', type=str, choices = ['cue_accumulation'], default='cue_accumulation', help='Choice of the dataset')
    parser.add_argument('--shuffle', type=bool, default=True, help='Enables shuffling sample order in datasets after each epoch')
    parser.add_argument('--trials', type=int, default=1, help='Nomber of trial experiments to do (i.e. repetitions with different initializations)')
    parser.add_argument('--epochs', type=int, default=10, help='Number of epochs to train')
    parser.add_argument('--optimizer', type=str, choices = ['SGD', 'NAG', 'Adam', 'RMSProp'], default='Adam', help='Choice of the optimizer')
    parser.add_argument('--loss', type=str, choices = ['MSE', 'BCE', 'CE'], default='BCE', help='Choice of the loss function (only for performance monitoring purposes, does not influence learning)')
    parser.add_argument('--lr', type=float, default=1e-4, help='Initial learning rate')
    parser.add_argument('--lr-layer-norm', type=float, nargs='+', default=(0.05,0.05,1.0), help='Per-layer modulation factor of the learning rate')
    parser.add_argument('--batch-size', type=int, default=5, help='Batch size for training (limited by the available GPU memory)')
    parser.add_argument('--test-batch-size', type=int, default=5, help='Batch size for testing (limited by the available GPU memory)')
    parser.add_argument('--train-len', type=int, default=80, help='Number of training set samples')
    parser.add_argument('--test-len', type=int, default=20, help='Number of test set samples')
    parser.add_argument('--visualize', type=bool, default=True, help='Enable network visualization')
    parser.add_argument('--visualize-light', type=bool, default=True, help='Enable light mode in network visualization, plots traces only for a single neuron')
    # Network model parameters
    parser.add_argument('--n_rec', type=int, default=100, help='Number of recurrent units')
    parser.add_argument('--model', type=str, choices = ['LIF'], default='LIF', help='Neuron model in the recurrent layer. Support for the ALIF neuron model has been removed.')
    parser.add_argument('--threshold', type=float, default=0.6, help='Firing threshold in the recurrent layer')
    parser.add_argument('--tau-mem', type=float, default=2000e-3, help='Membrane potential leakage time constant in the recurrent layer (in seconds)')
    parser.add_argument('--tau-out', type=float, default=20e-3, help='Membrane potential leakage time constant in the output layer (in seconds)')
    parser.add_argument('--bias-out', type=float, default=0.0, help='Bias of the output layer')
    parser.add_argument('--gamma', type=float, default=0.3, help='Surrogate derivative magnitude parameter')
    parser.add_argument('--w-init-gain', type=float, nargs='+', default=(0.5,0.1,0.5), help='Gain parameter for the He Normal initialization of the input, recurrent and output layer weights')
    

    my_args = parser.parse_args()

    return my_args

In [8]:
import train
import setup
(device, train_loader, traintest_loader, test_loader) = setup.setup(args) 

AttributeError: 'function' object has no attribute 'cpu'

In [ ]:

for batch_idx, (data, label) in enumerate(loader):

In [13]:
#from utilis import spike_time_to_spike 
def spike_time_to_spike(spike_times_up_list, spike_times_dn_list, trial_len=3000):
    nbsamples=len(spike_times_dn_list)
    nbchannels=len(spike_times_up_list[0])
    spikes=np.zeros((nbsamples, trial_len, nbchannels*2))
    for sample in range(nbsamples):
        for channel in range(nbchannels):
            nbspikes=len(spike_times_up_list[sample][channel])
            for t in range(nbspikes):
                index=spike_times_up_list[sample][channel][t]
                spikes[sample,index,channel]=1

    for sample in range(nbsamples):
        for channel in range(nbchannels):
            nbspikes=len(spike_times_dn_list[sample][channel])
            for t in range(nbspikes):
                index=spike_times_dn_list[sample][channel][t]
                spikes[sample,index,channel+nbchannels]=1

    

    return spikes


In [ ]:
pip install numpy

In [14]:
import numpy as np

data=np.load("dataset/bci3_encodedbci31111.npz", allow_pickle=True)
spike_time_up=data["spike_times_train_up"]
spike_time_dn=data["spike_times_train_dn"]
spikes=spike_time_to_spike(spike_time_up, spike_time_dn, 3000)
print(spikes)



MemoryError: Unable to allocate 814. MiB for an array with shape (278, 3000, 128) and data type float64